In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import wandb
import os, sys
import glob
import numpy as np
import torch
import pandas as pd
import random
import torch.nn as nn
import pickle
import torch.nn.functional as F

In [3]:
from mstcn_model import *
from utility.adaptive_data_loader import Breakfast, collate_fn_override
from utils import calculate_mof, dotdict

In [4]:
os.environ["WANDB_API_KEY"] = "992b3b1371ba79f48484cfca522b3786d7fa52c2"
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dipika_singhania (use `wandb login --relogin` to force relogin)


True

In [5]:
seed = 42

# Ensure deterministic behavior
def set_seed():
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
set_seed()

# Device configuration
os.environ['CUDA_VISIBLE_DEVICES']='6'
# os.environ['CUDA_LAUNCH_BLOCKING']='6'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
config = dotdict(
    epochs=500,
    num_class=48,
    batch_size=8,
    learning_rate=5e-4,
    weight_decay=0,
    dataset="Breakfast",
    architecture="unet-ensemble",
    features_file_name="/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/features/",
    chunk_size=1,
    max_frames_per_video=1200,
    feature_size=2048,
    ground_truth_files_dir="/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/groundTruth/",
    label_id_csv="/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/mapping.csv",
    gamma=0.1,
    step_size=500,
    split=1,
#     output_dir="/mnt/data/ar-datasets/dipika/breakfast/ms_tcn/data/breakfast/results/unsuper-finetune-split2-0.05-data-llr/",
    output_dir="/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast//results/mstcnnew-full-supervised-split1/",
    project_name="breakfast-split-1",
    train_split_file="/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/splits/train.split{}.bundle",
    test_split_file="/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/splits/test.split{}.bundle",
    all_files="/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/splits/all_files.txt",
    cutoff=8,
    data_per = 0.2,
    budget=40,
    semi_supervised_split="/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/semi_supervised/train.split{}_amt{}.bundle")

config.train_split_file = config.train_split_file.format(config.split)
config.semi_supervised_split = config.semi_supervised_split.format(config.split, config.data_per)
config.test_split_file = config.test_split_file.format(config.split)

if not os.path.exists(config.output_dir):
    os.mkdir(config.output_dir)

print(config)

{'epochs': 500, 'num_class': 48, 'batch_size': 8, 'learning_rate': 0.0005, 'weight_decay': 0, 'dataset': 'Breakfast', 'architecture': 'unet-ensemble', 'features_file_name': '/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/features/', 'chunk_size': 1, 'max_frames_per_video': 1200, 'feature_size': 2048, 'ground_truth_files_dir': '/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/groundTruth/', 'label_id_csv': '/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/mapping.csv', 'gamma': 0.1, 'step_size': 500, 'split': 1, 'output_dir': '/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast//results/mstcnnew-full-supervised-split1/', 'project_name': 'breakfast-split-1', 'train_split_file': '/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/splits/train.split1.bundle', 'test_split_file': '/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/splits/test.split1.bundle', 'all_files': '/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast/splits/all_files.txt', 'cutoff': 8, 'data_per': 0.2, 'budget': 40, 'semi_supervised_

In [7]:
traindataset = Breakfast(config, fold='train', fold_file_name=config.train_split_file)
testdataset = Breakfast(config, fold='test', fold_file_name=config.test_split_file)

Number of videos logged in train fold is 1460
Number of videos not found in train fold is 0
Number of videos logged in test fold is 252
Number of videos not found in test fold is 0


In [8]:
def _init_fn(worker_id):
    np.random.seed(int(seed))
trainloader = torch.utils.data.DataLoader(dataset=traindataset,
                                          batch_size=config.batch_size, 
                                          shuffle=True,
                                          pin_memory=True, num_workers=4, 
                                          collate_fn=lambda x: collate_fn_override(x, config.max_frames_per_video),
                                          worker_init_fn=_init_fn)
testloader = torch.utils.data.DataLoader(dataset=testdataset,
                                          batch_size=config.batch_size, 
                                          shuffle=False,
                                          pin_memory=True, num_workers=4,
                                          collate_fn=lambda x: collate_fn_override(x, config.max_frames_per_video),
                                          worker_init_fn=_init_fn)

In [18]:
set_seed()
model = MultiStageModel(num_stages=4, num_layers=10, num_f_maps=64, dim=2048, num_classes=48).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

# Requires loaded_vidid_selected_frames, boundaries_dict
ce_criterion = nn.CrossEntropyLoss(ignore_index=-100)
mse_criterion = nn.MSELoss(reduction='none')

In [19]:
def get_single_random(labels_all, video_ids):
    # Generate target for only timestamps. Do not generate pseudo labels at first 30 epochs.
    boundary_target_tensor = torch.ones((labels_all.shape[0], labels_all.shape[1]), dtype=torch.long, device=labels_all.device) * (-100)
    for iter_num, labels in enumerate(labels_all):
        cur_vidid = video_ids[iter_num]
        frame_idx_tensor = torch.from_numpy(np.array(loaded_vidid_selected_frames[cur_vidid + ".txt"]))
        boundary_target_tensor[iter_num, frame_idx_tensor] = labels[frame_idx_tensor]

    return boundary_target_tensor

In [21]:
initialize_epoch = 15
expectation_cal_gap = 5
best_val_acc = 0
for epoch in range(100):
    print("Starting Training")
    model.train()
    for i, item in enumerate(trainloader):
        item_0 = item[0].to(device)
        item_1 = item[1].to(device)
        item_2 = item[2].to(device)
        src_mask = torch.arange(item_2.shape[1], device=item_2.device)[None, :] < item_1[:, None]
        src_mask_mse = src_mask.unsqueeze(1).to(torch.float32).to(device)
        optimizer.zero_grad()
        
        middle_pred, predictions = model(item_0, src_mask_mse)
        
        loss = 0
        for p in predictions:
            loss += ce_criterion(p, item_2)
            loss += 0.15 * torch.mean(torch.clamp(mse_criterion(F.log_softmax(p[:, :, 1:], dim=1), 
                                                                F.log_softmax(p.detach()[:, :, :-1], dim=1)), min=0,
                                        max=16) * src_mask_mse[:, :, 1:])
            

        loss.backward()
        optimizer.step()
        
        if i % 10 == 0:
            with torch.no_grad():
                pred = torch.argmax(predictions[-1], dim=1)
                correct = float(torch.sum((pred == item_2) * src_mask).item())
                total = float(torch.sum(src_mask).item())
                print(f"Training:: Epoch {epoch}, Iteration {i}, Current loss {loss.item()}" +
                      f" Accuracy {correct * 100.0 / total}")
    # Calculating Expectation Step
    model.eval()

    print("Calculating Validation Data Accuracy")
    correct = 0.0
    total = 0.0
    for i, item in enumerate(testloader):
        with torch.no_grad():
            item_0 = item[0].to(device)
            item_1 = item[1].to(device)
            item_2 = item[2].to(device)
            src_mask = torch.arange(item_2.shape[1], device=item_2.device)[None, :] < item_1[:, None]
            src_mask_mse = src_mask.unsqueeze(1).to(torch.float32).to(device)

            middle_pred, predictions = model(item_0, src_mask_mse)

            pred = torch.argmax(predictions[-1], dim=1)
            correct += float(torch.sum((pred == item_2) * src_mask).item())
            total += float(torch.sum(src_mask).item())
    val_acc = correct * 100.0 / total
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), config.output_dir + "ms-tcn-best-model.wt")
    torch.save(model.state_dict(), config.output_dir + "ms-tcn-last-model.wt")
    print(f"Validation:: Epoch {epoch}, Probability Accuracy {val_acc}")

Starting Training
Training:: Epoch 0, Iteration 0, Current loss 15.756733894348145 Accuracy 3.641727949271214
Training:: Epoch 0, Iteration 10, Current loss 14.47412109375 Accuracy 5.982818919561305
Training:: Epoch 0, Iteration 20, Current loss 14.68253231048584 Accuracy 4.0693053568590045
Training:: Epoch 0, Iteration 30, Current loss 14.885210037231445 Accuracy 2.561191016906384
Training:: Epoch 0, Iteration 40, Current loss 13.509674072265625 Accuracy 5.475828318232921
Training:: Epoch 0, Iteration 50, Current loss 12.238069534301758 Accuracy 12.148139654875882
Training:: Epoch 0, Iteration 60, Current loss 11.681089401245117 Accuracy 14.903366468349285
Training:: Epoch 0, Iteration 70, Current loss 14.398063659667969 Accuracy 11.666864678626588
Training:: Epoch 0, Iteration 80, Current loss 11.45858097076416 Accuracy 19.09746781723016
Training:: Epoch 0, Iteration 90, Current loss 12.438542366027832 Accuracy 9.97188085815455
Training:: Epoch 0, Iteration 100, Current loss 11.08060

Training:: Epoch 4, Iteration 80, Current loss 7.860737323760986 Accuracy 48.83270991874113
Training:: Epoch 4, Iteration 90, Current loss 5.388908386230469 Accuracy 67.75573287326576
Training:: Epoch 4, Iteration 100, Current loss 5.907341003417969 Accuracy 59.64143694817012
Training:: Epoch 4, Iteration 110, Current loss 5.618639945983887 Accuracy 58.89663182346109
Training:: Epoch 4, Iteration 120, Current loss 4.5089006423950195 Accuracy 65.88147317255562
Training:: Epoch 4, Iteration 130, Current loss 5.96887731552124 Accuracy 48.37704639438056
Training:: Epoch 4, Iteration 140, Current loss 5.892844200134277 Accuracy 44.78202060139424
Training:: Epoch 4, Iteration 150, Current loss 6.27905797958374 Accuracy 48.41649238123693
Training:: Epoch 4, Iteration 160, Current loss 7.4477386474609375 Accuracy 33.881813550687724
Training:: Epoch 4, Iteration 170, Current loss 5.676479339599609 Accuracy 51.22993537627684
Training:: Epoch 4, Iteration 180, Current loss 5.840364456176758 Accur

Training:: Epoch 8, Iteration 160, Current loss 4.70130729675293 Accuracy 66.35419019652134
Training:: Epoch 8, Iteration 170, Current loss 4.066490173339844 Accuracy 73.32313485526007
Training:: Epoch 8, Iteration 180, Current loss 4.246727466583252 Accuracy 70.22984522984522
Calculating Validation Data Accuracy
Validation:: Epoch 8, Probability Accuracy 57.74263882458619
Starting Training
Training:: Epoch 9, Iteration 0, Current loss 3.986987352371216 Accuracy 71.22844827586206
Training:: Epoch 9, Iteration 10, Current loss 3.6933815479278564 Accuracy 78.65654791771199
Training:: Epoch 9, Iteration 20, Current loss 3.883115768432617 Accuracy 75.3448587434492
Training:: Epoch 9, Iteration 30, Current loss 4.349427223205566 Accuracy 67.16570261993876
Training:: Epoch 9, Iteration 40, Current loss 3.6414315700531006 Accuracy 70.61312646008727
Training:: Epoch 9, Iteration 50, Current loss 5.494977951049805 Accuracy 56.661772367735516
Training:: Epoch 9, Iteration 60, Current loss 5.9032

Training:: Epoch 13, Iteration 30, Current loss 3.8657870292663574 Accuracy 67.54674433900216
Training:: Epoch 13, Iteration 40, Current loss 3.4667370319366455 Accuracy 77.7344359494461
Training:: Epoch 13, Iteration 50, Current loss 3.1488115787506104 Accuracy 82.01793959980893
Training:: Epoch 13, Iteration 60, Current loss 2.5655517578125 Accuracy 83.9131340485366
Training:: Epoch 13, Iteration 70, Current loss 3.2787179946899414 Accuracy 79.36426993030767
Training:: Epoch 13, Iteration 80, Current loss 2.7792625427246094 Accuracy 85.11021421918659
Training:: Epoch 13, Iteration 90, Current loss 3.193089246749878 Accuracy 80.6383569247849
Training:: Epoch 13, Iteration 100, Current loss 2.5222582817077637 Accuracy 87.60703019378099
Training:: Epoch 13, Iteration 110, Current loss 3.5883288383483887 Accuracy 70.30478255631851
Training:: Epoch 13, Iteration 120, Current loss 2.9537813663482666 Accuracy 79.2634884384813
Training:: Epoch 13, Iteration 130, Current loss 2.99989128112792

Training:: Epoch 17, Iteration 100, Current loss 1.8833720684051514 Accuracy 92.32432432432432
Training:: Epoch 17, Iteration 110, Current loss 2.717165470123291 Accuracy 86.87916975537435
Training:: Epoch 17, Iteration 120, Current loss 2.713301658630371 Accuracy 83.67258820626697
Training:: Epoch 17, Iteration 130, Current loss 3.3141119480133057 Accuracy 79.44361352078971
Training:: Epoch 17, Iteration 140, Current loss 4.216887950897217 Accuracy 70.3354512536722
Training:: Epoch 17, Iteration 150, Current loss 3.26364803314209 Accuracy 76.44604316546763
Training:: Epoch 17, Iteration 160, Current loss 4.662065505981445 Accuracy 65.59694492831301
Training:: Epoch 17, Iteration 170, Current loss 2.435891628265381 Accuracy 89.90437660904745
Training:: Epoch 17, Iteration 180, Current loss 2.060638427734375 Accuracy 91.60161760556757
Calculating Validation Data Accuracy
Validation:: Epoch 17, Probability Accuracy 68.4978097510595
Starting Training
Training:: Epoch 18, Iteration 0, Curr

...

Training:: Epoch 82, Iteration 100, Current loss 0.5890763401985168 Accuracy 98.83960321916527
Training:: Epoch 82, Iteration 110, Current loss 0.62496018409729 Accuracy 99.19314170448816
Training:: Epoch 82, Iteration 120, Current loss 0.673930287361145 Accuracy 98.98267980168694
Training:: Epoch 82, Iteration 130, Current loss 0.6009506583213806 Accuracy 98.74810318664643
Training:: Epoch 82, Iteration 140, Current loss 0.6075416207313538 Accuracy 98.86310124453414
Training:: Epoch 82, Iteration 150, Current loss 0.45045557618141174 Accuracy 99.19159256265158
Training:: Epoch 82, Iteration 160, Current loss 0.47594520449638367 Accuracy 99.42282881781762
Training:: Epoch 82, Iteration 170, Current loss 0.4625689685344696 Accuracy 98.99150743099787
Training:: Epoch 82, Iteration 180, Current loss 0.664405882358551 Accuracy 98.78590464909684
Calculating Validation Data Accuracy
Validation:: Epoch 82, Probability Accuracy 69.42337294379746
Starting Training
Training:: Epoch 83, Iteration

Training:: Epoch 86, Iteration 170, Current loss 0.5418944358825684 Accuracy 99.15644923982344
Training:: Epoch 86, Iteration 180, Current loss 0.5177955627441406 Accuracy 99.2443857331572
Calculating Validation Data Accuracy
Validation:: Epoch 86, Probability Accuracy 69.30505597302847
Starting Training
Training:: Epoch 87, Iteration 0, Current loss 0.4864428639411926 Accuracy 99.13434903047091
Training:: Epoch 87, Iteration 10, Current loss 0.5609903335571289 Accuracy 98.51579833218575
Training:: Epoch 87, Iteration 20, Current loss 0.5465987324714661 Accuracy 99.15722797788395
Training:: Epoch 87, Iteration 30, Current loss 0.47308349609375 Accuracy 98.8706546673725
Training:: Epoch 87, Iteration 40, Current loss 0.4479682147502899 Accuracy 99.31270096463022
Training:: Epoch 87, Iteration 50, Current loss 0.4842802882194519 Accuracy 99.07993099482461
Training:: Epoch 87, Iteration 60, Current loss 0.4824381470680237 Accuracy 98.97510980966325
Training:: Epoch 87, Iteration 70, Curre

Training:: Epoch 91, Iteration 30, Current loss 1.0162017345428467 Accuracy 96.84556734937958
Training:: Epoch 91, Iteration 40, Current loss 0.6875038743019104 Accuracy 97.44475138121547
Training:: Epoch 91, Iteration 50, Current loss 0.7617689371109009 Accuracy 98.71574835769663
Training:: Epoch 91, Iteration 60, Current loss 0.8581153750419617 Accuracy 97.67405638336535
Training:: Epoch 91, Iteration 70, Current loss 0.6955350637435913 Accuracy 98.40155615696888
Training:: Epoch 91, Iteration 80, Current loss 0.9384129643440247 Accuracy 97.41739460691227
Training:: Epoch 91, Iteration 90, Current loss 0.7438524961471558 Accuracy 98.66089822826535
Training:: Epoch 91, Iteration 100, Current loss 0.7553794980049133 Accuracy 98.30028328611898
Training:: Epoch 91, Iteration 110, Current loss 0.9200969934463501 Accuracy 97.66350372410979
Training:: Epoch 91, Iteration 120, Current loss 0.9388884902000427 Accuracy 97.12379702537183
Training:: Epoch 91, Iteration 130, Current loss 0.746585

Training:: Epoch 95, Iteration 90, Current loss 0.4715726673603058 Accuracy 98.99235872029557
Training:: Epoch 95, Iteration 100, Current loss 0.5600777864456177 Accuracy 99.29146211852827
Training:: Epoch 95, Iteration 110, Current loss 0.5836320519447327 Accuracy 98.9923533195169
Training:: Epoch 95, Iteration 120, Current loss 0.539239764213562 Accuracy 99.0307037203048
Training:: Epoch 95, Iteration 130, Current loss 0.5439128279685974 Accuracy 99.3272026506187
Training:: Epoch 95, Iteration 140, Current loss 0.5546233057975769 Accuracy 98.97136716051278
Training:: Epoch 95, Iteration 150, Current loss 0.4327958822250366 Accuracy 99.35102716734548
Training:: Epoch 95, Iteration 160, Current loss 0.5355766415596008 Accuracy 99.05660377358491
Training:: Epoch 95, Iteration 170, Current loss 0.6079598069190979 Accuracy 98.92939422686001
Training:: Epoch 95, Iteration 180, Current loss 0.4897409677505493 Accuracy 99.1116109080686
Calculating Validation Data Accuracy
Validation:: Epoch 

Training:: Epoch 99, Iteration 150, Current loss 1.1199856996536255 Accuracy 98.0381703806376
Training:: Epoch 99, Iteration 160, Current loss 1.2395108938217163 Accuracy 95.79732914375491
Training:: Epoch 99, Iteration 170, Current loss 0.550856351852417 Accuracy 98.35824037044833
Training:: Epoch 99, Iteration 180, Current loss 1.096672534942627 Accuracy 96.35301895107978
Calculating Validation Data Accuracy
Validation:: Epoch 99, Probability Accuracy 69.60005698208626


In [24]:
!ls '/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast//results/em-maximize-mstcn-split3/ms-tcn-initial-15-epochs.wt'


/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast//results/em-maximize-mstcn-split3/ms-tcn-initial-15-epochs.wt


In [24]:
torch.save(model.state_dict(),
"/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast//results/em-maximize-mstcn-speed/final-em-maximized.wt")

In [34]:
model.load_state_dict(torch.load(f"/mnt/ssd/all_users/dipika/ms_tcn/data/breakfast//results/em-maximize-mstcn-split3/ms-tcn-initial-15-epochs.wt"))

<All keys matched successfully>

In [25]:
print("Calculating Validation Data Accuracy")
correct = 0.0
total = 0.0
for i, item in enumerate(testloader):
    with torch.no_grad():
        item_0 = item[0].to(device)
        item_1 = item[1].to(device)
        item_2 = item[2].to(device)
        src_mask = torch.arange(item_2.shape[1], device=item_2.device)[None, :] < item_1[:, None]
        src_mask_mse = src_mask.unsqueeze(1).to(torch.float32).to(device)

        middle_pred, predictions = model(item_0, src_mask_mse)

        pred = torch.argmax(predictions[-1], dim=1)
        correct += float(torch.sum((pred == item_2) * src_mask).item())
        total += float(torch.sum(src_mask).item())

print(f"Validation:: Epoch {epoch}, Probability Accuracy {correct * 100.0 / total}")

Calculating Validation Data Accuracy
Validation:: Epoch 15, Probability Accuracy 57.770101729343025


In [9]:
import pickle
# pickle.dump(video_id_boundary_frames, open("dump_dir/video_id_boundary_frames_dict.pkl", "wb"))
# pickle.dump(loaded_vidid_selected_frames, open("dump_dir/loaded_vidid_selected_frames_dict.pkl", "wb"))
pickle.dump(boundary_dict, open("dump_dir/chunk_1_video_id_boundary_frames_dict.pkl", "wb"))